# Optimizing the Python Code for Big Data 
Balancing Coding Complexity against Computational Complexity 

    
    AUTHOR: Dr. Roy Jafari 

# Chapter 5: Picking up the right tool 

## Challenge 3: Restructuring and Reformulating Data – Second Case Study

In this challenge, we address a common issue in data preparation. Often, data is stored in formats that either omit zero-value objects or use lists or dictionaries within individual table cells to save disk space. While this method is efficient for storage, it can significantly complicate data preparation. This learning opportunity allows us to tackle these challenges and explore optimized tools for handling them. Follow the nine steps below to familiarize yourself with these challenges and learn how to manage them effectively.

1. The code below uses `pd.read_csv()` to load the `stock_news.csv` file into the `news_df` DataFrame. Execute the code and review the dataset:

```python
import pandas as pd
news_df = pd.read_csv('stock_news.csv')
news_df
```

In [21]:
import pandas as pd
news_df = pd.read_csv('stock_news.csv')
news_df

,Unnamed: 0,Title,DateTime,Positive,Negative,Neutral,Entities,DateTime_Hour,Ticker
0,0,"Market Tumbles as VCLT, SPY, and CMCSA Stocks ...",2023-01-01 01:18:00,0.05,0.78,0.17,"['VCLT', 'SPY', 'CMCSA']",2023-01-01 01:00:00,"['VCLT', 'SPY', 'CMCSA']"
1,1,Tech stocks tumble as QQQ and META hit new low...,2023-01-01 01:27:00,0.24,0.26,0.50,"['QQQ', 'SCHP', 'META', 'BG', 'APA', 'YXI', 'V...",2023-01-01 01:00:00,"['QQQ', 'SCHP', 'META', 'BG', 'APA', 'YXI', 'V..."
2,2,Tech Giant NVDA Faces Lawsuit Over Patent Infr...,2023-01-01 01:40:00,0.03,0.72,0.25,"['NVDA', 'XMLV', 'SJB', 'FLOT', 'AIVL', 'SPY',...",2023-01-01 01:00:00,"['NVDA', 'XMLV', 'SJB', 'FLOT', 'AIVL', 'SPY',..."
3,3,FAB Inc. announces record-breaking revenue but...,2023-01-01 01:56:00,0.25,0.06,0.69,['FAB'],2023-01-01 01:00:00,['FAB']
4,4,Tech Giants MSFT and XLK Experience Minor Down...,2023-01-01 02:06:00,0.20,0.03,0.77,"['EUM', 'PCY', 'SPHQ', 'SUSA', 'EPV', 'SHV', '...",2023-01-01 02:00:00,"['EUM', 'PCY', 'SPHQ', 'SUSA', 'EPV', 'SHV', '..."
...,...,...,...,...,...,...,...,...,...
38265,38265,Oil prices plunge leading to major losses for ...,2023-12-29 23:01:00,0.01,0.97,0.02,"['SWK', 'META', 'NWL', 'OKE', 'DIG', 'EDZ']",2023-12-29 23:00:00,"['SWK', 'META', 'NWL', 'OKE', 'DIG', 'EDZ']"
38266,38266,Tech Giants Face Regulatory Scrutiny Leading t...,2023-12-29 23:08:00,0.23,0.52,0.25,"['AMZN', 'SPY', 'BZQ', 'DUG']",2023-12-29 23:00:00,"['AMZN', 'SPY', 'BZQ', 'DUG']"
38267,38267,"META surges but SPY struggles, investors conce...",2023-12-29 23:18:00,0.11,0.64,0.25,"['META', 'SPY']",2023-12-29 23:00:00,"['META', 'SPY']"
38268,38268,XMPT and FDX stocks soar while SPY takes a hit,2023-12-29 23:31:00,0.10,0.79,0.11,"['XMPT', 'SPY', 'FDX']",2023-12-29 23:00:00,"['XMPT', 'SPY', 'FDX']"


2. The following code confirms that this data belongs to the year 2023.

```python
news_df.DateTime = pd.to_datetime(news_df['DateTime'])
print(news_df.DateTime.min())
print(news_df.DateTime.max())
```

In [22]:
news_df.DateTime = pd.to_datetime(news_df['DateTime'])
print(news_df.DateTime.min())
print(news_df.DateTime.max())

2023-01-01 01:18:00
2023-12-29 23:40:00


3. Write a code snippet that extracts a unique list of all the stocks mentioned in the 'Entity' column of your dataframe. Challenge yourself to use the most efficient tools and techniques you can think of. Do not proceed to the next steps until you have completed this task. Once you are done, I will demonstrate the optimal tools and methods you could have used.

**Answer:**
I will do this with the wrong tools in this part, now.

In [23]:
%%time
all_stocks = []

for entities in news_df.Entities:
    entities_list = (entities[1:-1]
        .replace("'", "")
        .replace(" ", "")
        .split(',')
         )
    for entity in entities_list:
        if entity not in all_stocks:
            all_stocks.append(entity)
print(f"There are {len(all_stocks)} different stocks in news_df.")

There are 612 different stocks in news_df.
CPU times: user 427 ms, sys: 1.95 ms, total: 429 ms
Wall time: 434 ms


4. The code below addresses the challenge described in Step 3, utilizing the following tools: the `.str` accessor, `.replace()`, `.split()`, and `.explode()` functions from pandas, along with Python's `set()` and `list()` functions. Execute the code and compare its runtime with the method you previously implemented.

```python
all_stocks = list(
    set(
        news_df.Entities
        .str[1:-1]
        .str.replace("'", "")
        .str.replace(" ", "")
        .str.split(',')
        .explode()
        .values.tolist()
    )
)
print(f"There are {len(all_stocks)} different stocks in news_df.")
all_stocks
```

In [24]:
%%time
all_stocks = list(
    set(
        news_df.Entities
        .str[1:-1]
        .str.replace("'", "")
        .str.replace(" ", "")
        .str.split(',')
        .explode()
        .values.tolist()
    )
)
print(f"There are {len(all_stocks)} different stocks in news_df.")

There are 612 different stocks in news_df.
CPU times: user 74.3 ms, sys: 5.82 ms, total: 80.1 ms
Wall time: 82 ms


**Answer:**
The code that used for loops and the `.append()` function of Python lists took 434 ms, while the provided code took 82 ms. There was a meaningful improvement even with a data size of only 38,270 rows.

5. Each row of `news_df` represents one stock news. We want to use this dataset to create `sentiment_df`, where each row will represent the aggregate sentiment scores of each stock for each hour. The following code creates a Pandas DataFrame that is an empty version of `sentiment_df`. Run the following code to create `sentiment_df` and study its structure.

```python
import datetime

# Ensure the DateTime column is in datetime format
news_df['DateTime'] = pd.to_datetime(news_df['DateTime'])

# Generate all hours for the year 2023
all_hours = [
    datetime.datetime(2023, 1, 1) + datetime.timedelta(hours=i) 
    for i in range(365 * 24)]

# Create a MultiIndex using the list of stocks and all hours
my_multi_index = pd.MultiIndex.from_product(
    (all_stocks, all_hours), 
    names=['Ticker', 'DateTime'])

# Create an empty DataFrame with the specified MultiIndex and columns for sentiment scores
sentiment_df = pd.DataFrame(
    index=my_multi_index, 
    columns=['Positive', 'Negative', 'Neutral','n_news'])

# Display the structure of the empty DataFrame
sentiment_df
```

In [25]:
import datetime

# Ensure the DateTime column is in datetime format
news_df['DateTime'] = pd.to_datetime(news_df['DateTime'])

# Generate all hours for the year 2023
all_hours = [datetime.datetime(2023, 1, 1) + datetime.timedelta(hours=i) for i in range(365 * 24)]

# Create a MultiIndex using the list of stocks and all hours
my_multi_index = pd.MultiIndex.from_product((all_stocks, all_hours), names=['Ticker', 'DateTime'])

# Create an empty DataFrame with the specified MultiIndex and columns for sentiment scores
sentiment_df = pd.DataFrame(index=my_multi_index, columns=['Positive', 'Negative', 'Neutral','n_news'])

# Display the structure of the empty DataFrame
sentiment_df

Positive Negative Neutral n_news
Ticker DateTime                                            
XLE    2023-01-01 00:00:00      NaN      NaN     NaN    NaN
       2023-01-01 01:00:00      NaN      NaN     NaN    NaN
       2023-01-01 02:00:00      NaN      NaN     NaN    NaN
       2023-01-01 03:00:00      NaN      NaN     NaN    NaN
       2023-01-01 04:00:00      NaN      NaN     NaN    NaN
...                             ...      ...     ...    ...
SCHG   2023-12-31 19:00:00      NaN      NaN     NaN    NaN
       2023-12-31 20:00:00      NaN      NaN     NaN    NaN
       2023-12-31 21:00:00      NaN      NaN     NaN    NaN
       2023-12-31 22:00:00      NaN      NaN     NaN    NaN
       2023-12-31 23:00:00      NaN      NaN     NaN    NaN

[5361120 rows x 4 columns]

6. Your challenge in this step is to create the complete version of the `sentiment_df` DataFrame. Ensure you choose the most efficient tools and techniques to accomplish this task. Pay attention to whether you've picked the best tools to get this done. Once you are done, I will demonstrate the optimal tools and methods you could have used.

**Answer:**
I will do this with the wrong tools in this part, now.

In [26]:
news_df['DateTime_Hour'] = news_df.DateTime.apply(lambda v: datetime.datetime(v.year, v.month, v.day, v.hour, 0, 0))

In [27]:
%%time
import numpy as np

for i,row in news_df.iterrows():
    entities_list = (row.Entities[1:-1]
        .replace("'", "")
        .replace(" ", "")
        .split(',')
         )
    for entity in entities_list:
        wsr = sentiment_df.loc[(entity,row.DateTime_Hour)]
        if np.isnan(wsr.n_news):
            sentiment_df.loc[(entity,row.DateTime_Hour),'Positive'] = row.Positive
            sentiment_df.loc[(entity,row.DateTime_Hour),'Negative'] = row.Negative
            sentiment_df.loc[(entity,row.DateTime_Hour),'Neutral'] = row.Neutral
            sentiment_df.loc[(entity,row.DateTime_Hour),'n_news'] = 1
        else:
            current_positive = sentiment_df.loc[(entity,row.DateTime_Hour),'Positive']
            current_negative = sentiment_df.loc[(entity,row.DateTime_Hour),'Negative']
            current_neutral = sentiment_df.loc[(entity,row.DateTime_Hour),'Neutral']
            current_n_news = sentiment_df.loc[(entity,row.DateTime_Hour),'n_news']

            future_positive = ((current_positive * current_n_news) + row.Positive) / (current_n_news+1)
            future_negative = ((current_negative * current_n_news) + row.Negative) / (current_n_news+1)
            future_neutral = ((current_neutral * current_n_news) + row.Neutral) / (current_n_news+1)

            sentiment_df.loc[(entity,row.DateTime_Hour),'Positive'] = future_positive
            sentiment_df.loc[(entity,row.DateTime_Hour),'Negative'] = future_negative
            sentiment_df.loc[(entity,row.DateTime_Hour),'Neutral'] = future_neutral
            sentiment_df.loc[(entity,row.DateTime_Hour),'n_news'] = current_n_news+1

sentiment_df

CPU times: user 38.6 s, sys: 44.7 ms, total: 38.7 s
Wall time: 38.8 s


Positive Negative Neutral n_news
Ticker DateTime                                            
XLE    2023-01-01 00:00:00      NaN      NaN     NaN    NaN
       2023-01-01 01:00:00      NaN      NaN     NaN    NaN
       2023-01-01 02:00:00      NaN      NaN     NaN    NaN
       2023-01-01 03:00:00      NaN      NaN     NaN    NaN
       2023-01-01 04:00:00      NaN      NaN     NaN    NaN
...                             ...      ...     ...    ...
SCHG   2023-12-31 19:00:00      NaN      NaN     NaN    NaN
       2023-12-31 20:00:00      NaN      NaN     NaN    NaN
       2023-12-31 21:00:00      NaN      NaN     NaN    NaN
       2023-12-31 22:00:00      NaN      NaN     NaN    NaN
       2023-12-31 23:00:00      NaN      NaN     NaN    NaN

[5361120 rows x 4 columns]

7. The following code completes the challenge described in the previous step by utilizing the `.explode()` and `.groupby()` functions in an optimized way. Compare the runtime of your own code with this optimized version:
```python
%%time
news_df['Ticker'] = (news_df.Entities
        .str[1:-1]
        .str.replace("'", "")
        .str.replace(" ", "")
        .str.split(','))
sentiment_df = (news_df
                .explode('Ticker')
                .groupby(['Ticker', 'DateTime_Hour'])
                [['Positive', 'Negative', 'Neutral']]
                .mean()
            )
sentiment_df['n_news'] = (news_df
                .explode('Ticker')
                .groupby(['Ticker', 'DateTime_Hour'])
                .size()
            )
sentiment_df
```

In [28]:
%%time
news_df['Ticker'] = (news_df.Entities
        .str[1:-1]
        .str.replace("'", "")
        .str.replace(" ", "")
        .str.split(','))
sentiment_df = (news_df
                .explode('Ticker')
                .groupby(['Ticker','DateTime_Hour'])
                [['Positive','Negative','Neutral']]
                .mean()
            )
sentiment_df['n_news'] = (news_df
                .explode('Ticker')
                .groupby(['Ticker','DateTime_Hour'])
                .size()
            )
sentiment_df

CPU times: user 188 ms, sys: 13.4 ms, total: 201 ms
Wall time: 206 ms


Positive  Negative  Neutral  n_news
Ticker DateTime_Hour                                           
AAPL   2023-01-01 08:00:00      0.15      0.45     0.40       1
       2023-01-01 10:00:00      0.15      0.23     0.62       1
       2023-01-01 12:00:00      0.51      0.24     0.25       1
       2023-01-01 14:00:00      0.19      0.31     0.50       1
       2023-01-01 15:00:00      0.70      0.02     0.28       1
...                              ...       ...      ...     ...
ZSL    2023-12-26 18:00:00      0.90      0.05     0.05       1
       2023-12-27 12:00:00      0.12      0.63     0.25       1
       2023-12-27 19:00:00      0.15      0.36     0.49       1
       2023-12-27 23:00:00      0.02      0.96     0.02       1
       2023-12-28 12:00:00      0.56      0.19     0.25       1

[179056 rows x 4 columns]

**Answer:** Comparing the run time of this approach and the previous one shows the optimized approach makes the process 188 times faster.

8. The `sentiment_df` we get from the previous step does not have the exact data structure as the one described in step 5. Essentially, if during an hour we don't have any news for a ticker, the rows for that ticker have been omitted. Use the script that leverages what we completed in step 6, to ensure our `sentiment_df` will be complete. Pay attention to whether you've picked the best tools to get this done. Once you are done, I will demonstrate the optimal tools and methods you could have used.

**Answer:**
I will do this with the wrong tools in this part, now.

In [30]:
%%time
news_df['DateTime'] = pd.to_datetime(news_df['DateTime'])

all_hours = [datetime.datetime(2023, 1, 1) + datetime.timedelta(hours=i) for i in range(365 * 24)]
my_multi_index = pd.MultiIndex.from_product((all_stocks, all_hours), names=['Ticker', 'DateTime'])
stage_df = pd.DataFrame(index=my_multi_index, columns=['Positive', 'Negative', 'Neutral','n_news'])
stage_df.n_news = 0.0

for i, row in sentiment_df.iterrows():
    stage_df.loc[i] = row
sentiment_df = stage_df.copy()
sentiment_df


CPU times: user 3h 12min, sys: 2.18 s, total: 3h 12min 2s
Wall time: 3h 17min 55s


Positive Negative Neutral  n_news
Ticker DateTime                                             
XLE    2023-01-01 00:00:00      NaN      NaN     NaN     0.0
       2023-01-01 01:00:00      NaN      NaN     NaN     0.0
       2023-01-01 02:00:00      NaN      NaN     NaN     0.0
       2023-01-01 03:00:00      NaN      NaN     NaN     0.0
       2023-01-01 04:00:00      NaN      NaN     NaN     0.0
...                             ...      ...     ...     ...
SCHG   2023-12-31 19:00:00      NaN      NaN     NaN     0.0
       2023-12-31 20:00:00      NaN      NaN     NaN     0.0
       2023-12-31 21:00:00      NaN      NaN     NaN     0.0
       2023-12-31 22:00:00      NaN      NaN     NaN     0.0
       2023-12-31 23:00:00      NaN      NaN     NaN     0.0

[5361120 rows x 4 columns]

9. The following code snippet leverages the `.update()` function to address the challenge described in the previous step in an optimized manner. Please study and execute the code, then compare its runtime with your previous implementation:

```python
news_df['DateTime'] = pd.to_datetime(news_df['DateTime'])
all_hours = [
    datetime.datetime(2023, 1, 1) + 
    datetime.timedelta(hours=i) for i in range(365 * 24)]
my_multi_index = pd.MultiIndex.from_product(
    [all_stocks, all_hours], 
    names=['Ticker', 'DateTime_Hour'])
stage_df = pd.DataFrame(
    index=my_multi_index, 
    columns=['Positive', 'Negative', 'Neutral', 'n_news'])
stage_df['n_news'] = 0.0

# Update the DataFrame with values from sentiment_df
stage_df.update(sentiment_df)

# Copy the updated DataFrame to sentiment_df for further use
sentiment_df = stage_df.copy()

# Output the updated DataFrame
sentiment_df
```

In [33]:
%%time
news_df['DateTime'] = pd.to_datetime(news_df['DateTime'])
all_hours = [datetime.datetime(2023, 1, 1) + datetime.timedelta(hours=i) for i in range(365 * 24)]
my_multi_index = pd.MultiIndex.from_product([all_stocks, all_hours], names=['Ticker', 'DateTime_Hour'])
stage_df = pd.DataFrame(index=my_multi_index, columns=['Positive', 'Negative', 'Neutral', 'n_news'])
stage_df['n_news'] = 0.0

# Update the DataFrame with values from sentiment_df
stage_df.update(sentiment_df)

# Copy the updated DataFrame to sentiment_df for further use
sentiment_df = stage_df.copy()

# Output the updated DataFrame
sentiment_df

CPU times: user 3.69 s, sys: 240 ms, total: 3.93 s
Wall time: 3.99 s


Positive Negative Neutral  n_news
Ticker DateTime_Hour                                        
XLE    2023-01-01 00:00:00      NaN      NaN     NaN     0.0
       2023-01-01 01:00:00      NaN      NaN     NaN     0.0
       2023-01-01 02:00:00      NaN      NaN     NaN     0.0
       2023-01-01 03:00:00      NaN      NaN     NaN     0.0
       2023-01-01 04:00:00      NaN      NaN     NaN     0.0
...                             ...      ...     ...     ...
SCHG   2023-12-31 19:00:00      NaN      NaN     NaN     0.0
       2023-12-31 20:00:00      NaN      NaN     NaN     0.0
       2023-12-31 21:00:00      NaN      NaN     NaN     0.0
       2023-12-31 22:00:00      NaN      NaN     NaN     0.0
       2023-12-31 23:00:00      NaN      NaN     NaN     0.0

[5361120 rows x 4 columns]

**Answer**: We can see that the `for` loop method took much longer that `.update()` one. It was (3*60*60+17*60+55)/(4) = 2968.75 times faster. 

In [34]:
(3*60*60+17*60+55)/(4)

2968.75

10. For your future reference, it's useful to note the situations and tools from this challenge where optimization is most effective. The tools we used are `.explode()`, `pd.MultiIndex` and `.update()`.

**Answer**:
- **`.explode()`**
This function is ideal for handling DataFrame columns with list-like or nested values. It "explodes" these lists into separate rows, maintaining the context of the original row. Use it when you need to flatten a column of arrays or lists into a DataFrame where each element of the array or list gets its own row but retains the association with other column values.

- **`pd.MultiIndex`**
MultiIndex, or hierarchical indexing, is perfect for complex data analysis, enabling you to work with higher dimensional data using lower dimensional structures. It's particularly useful when you need to summarize data by multiple grouping variables, or when accessing subsets of a DataFrame is crucial to performance and clarity of your analysis.

- **`.update()`**
The `.update()` function is used to modify a DataFrame in place using another DataFrame. It is especially efficient when you want to update a subset of a DataFrame's values without rewriting the entire frame. This is useful in situations where only small parts of the dataset need changes which are already aligned in index or columns with the original DataFrame. It avoids the overhead of more computationally intensive operations like joins.
